# Prática Independente: Feature Engineering e validação de modelos.

### Feature Engineering.

#### Exercício 1: Acesse o link com a [lista das montanhas mais altas do mundo](https://en.wikipedia.org/wiki/List_of_highest_mountains_on_Earth) e crie um dicionário para as cinco primeiras, em termos de altura, contendo dados sobre `Mountain`, `Height`, `First Ascent` e `Country`.

In [1]:
import pandas as pd
import numpy as np

In [2]:
dados= {'mountain': ['Everest', 'K2', 'Kangchenjunga', 'Lhotse', 'Makalu'], 
        'height': [8.848, 8.611, 8.586, 8.516, 8.485], 
        'first_ascent': [1953, 1954, 1955, 1956, 1955], 
        'country': ['China', 'China', 'India', 'China', 'China']}

df= pd.DataFrame(dados, columns= dados.keys())
df.head()

,mountain,height,first_ascent,country
0,Everest,8.848,1953,China
1,K2,8.611,1954,China
2,Kangchenjunga,8.586,1955,India
3,Lhotse,8.516,1956,China
4,Makalu,8.485,1955,China


#### Exercício 2: Aplique a técnica `one hot-encoding` à lista de dicionários recém criados, para gerar a "dummiezação" das variáveis categóricas.

In [3]:
#Usando função get_dummies do pandas

df_dummies= pd.get_dummies(df)
df_dummies

,height,first_ascent,mountain_Everest,mountain_K2,mountain_Kangchenjunga,mountain_Lhotse,mountain_Makalu,country_China,country_India
0,8.848,1953,1,0,0,0,0,1,0
1,8.611,1954,0,1,0,0,0,1,0
2,8.586,1955,0,0,1,0,0,0,1
3,8.516,1956,0,0,0,1,0,1,0
4,8.485,1955,0,0,0,0,1,1,0


#### Exercício 3: Recupere os nomes dos atributos "dummiezados" e os inspecione.

In [4]:
df_dummies.describe()

,height,first_ascent,mountain_Everest,mountain_K2,mountain_Kangchenjunga,mountain_Lhotse,mountain_Makalu,country_China,country_India
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,8.609200,1954.600000,0.200000,0.200000,0.200000,0.200000,0.200000,0.800000,0.200000
std,0.142898,1.140175,0.447214,0.447214,0.447214,0.447214,0.447214,0.447214,0.447214
min,8.485000,1953.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.516000,1954.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,8.586000,1955.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,8.611000,1955.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,8.848000,1956.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
df_dummies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   height                  5 non-null      float64
 1   first_ascent            5 non-null      int64  
 2   mountain_Everest        5 non-null      uint8  
 3   mountain_K2             5 non-null      uint8  
 4   mountain_Kangchenjunga  5 non-null      uint8  
 5   mountain_Lhotse         5 non-null      uint8  
 6   mountain_Makalu         5 non-null      uint8  
 7   country_China           5 non-null      uint8  
 8   country_India           5 non-null      uint8  
dtypes: float64(1), int64(1), uint8(7)
memory usage: 243.0 bytes


#### Exercício 4: Calcule a matriz esparsa dos dados.

In [6]:
from scipy.sparse import csr_matrix
matriz_esparsa= csr_matrix(df_dummies).toarray()

matriz_esparsa

array([[8.848e+00, 1.953e+03, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 1.000e+00, 0.000e+00],
       [8.611e+00, 1.954e+03, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 1.000e+00, 0.000e+00],
       [8.586e+00, 1.955e+03, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 1.000e+00],
       [8.516e+00, 1.956e+03, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00,
        0.000e+00, 1.000e+00, 0.000e+00],
       [8.485e+00, 1.955e+03, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        1.000e+00, 1.000e+00, 0.000e+00]])

### Validação de modelos.

#### Exercício 5: A partir das definições a seguir, construa a validação dos modelos a serem aplicados para ajustar os pontos criados.

#### Exercício 5 (continuação): Teste os graus `[1, 3, 5, 7, 9]` para os pontos gerados.

#### Exercício 5 (continuação): Escolha o grau que melhor ajusta a regressão aos pontos.

#### Exercício 5 (continuação): Plote os pontos gerados e a curva de ajuste final.